In [1]:
from rasa_nlu.training_data import load_data
from rasa_nlu import config
from rasa_nlu.model import Trainer, Metadata, Interpreter

In [2]:
def train_nlu(data, configuration, model_dir):
    training_data = load_data(data)
    trainer = Trainer(config.load(configuration))
    trainer.train(training_data)
    model_directory = trainer.persist(model_dir,fixed_model_name='faq_chatbot_nlu')

In [3]:
train_nlu_model_switch = True

In [4]:
if (train_nlu_model_switch):
    train_nlu('./data/data.json','spacy_configuration.json','./models/faq_chatbot_nlu')

C:\Users\Utkarsh Sah\Anaconda3\lib\site-packages\rasa_nlu\training_data\training_data.py:186: UserWarning: Intent 'pio eligibility' has only 1 training examples! Minimum is 2, training may fail.
  self.MIN_EXAMPLES_PER_INTENT))


Fitting 2 folds for each of 6 candidates, totalling 12 fits


C:\Users\Utkarsh Sah\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=2.
  % (min_groups, self.n_splits)), Warning)
C:\Users\Utkarsh Sah\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Utkarsh Sah\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Utkarsh Sah\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, war

In [5]:
def run_nlu(query):
    interpreter = Interpreter.load('./models/faq_chatbot_nlu/default/faq_chatbot_nlu')
    return interpreter.parse(query)

In [6]:
query = "Thanks!"

In [7]:
parsed_query = run_nlu(query)
parsed_query

C:\Users\Utkarsh Sah\Anaconda3\lib\site-packages\rasa_nlu\extractors\entity_synonyms.py:85: UserWarning: Failed to load synonyms file from './models/faq_chatbot_nlu/default/faq_chatbot_nlu\entity_synonyms.json'
  "".format(entity_synonyms_file))


{'entities': [],
 'intent': {'confidence': 0.33110562231519619, 'name': 'gratitude'},
 'intent_ranking': [{'confidence': 0.33110562231519619, 'name': 'gratitude'},
  {'confidence': 0.30790794445649089, 'name': 'greet'},
  {'confidence': 0.088807057574621401, 'name': 'goodbye'},
  {'confidence': 0.045134071046979828, 'name': 'diplomatic network services'},
  {'confidence': 0.037512288521168731, 'name': 'pio definition'},
  {'confidence': 0.035896898035324865, 'name': 'nri definition'},
  {'confidence': 0.035743092405548391, 'name': 'high commissions purpose'},
  {'confidence': 0.034374256090003261, 'name': 'high commissions definiton'},
  {'confidence': 0.033915078726836974, 'name': 'nri eligibility'},
  {'confidence': 0.029808100358808098, 'name': 'diplomatic network size'}],
 'text': 'Thanks!'}

In [14]:
run_nlu("nigger")

C:\Users\Utkarsh Sah\Anaconda3\lib\site-packages\rasa_nlu\extractors\entity_synonyms.py:85: UserWarning: Failed to load synonyms file from './models/faq_chatbot_nlu/default/faq_chatbot_nlu\entity_synonyms.json'
  "".format(entity_synonyms_file))


{'entities': [],
 'intent': {'confidence': 0.28682415734080252, 'name': 'greet'},
 'intent_ranking': [{'confidence': 0.28682415734080252, 'name': 'greet'},
  {'confidence': 0.2147663723000526, 'name': 'nri definition'},
  {'confidence': 0.12593716622074003, 'name': 'goodbye'},
  {'confidence': 0.11457505903872228, 'name': 'pio definition'},
  {'confidence': 0.077975554955377918, 'name': 'nri eligibility'},
  {'confidence': 0.070450087281308291, 'name': 'diplomatic network size'},
  {'confidence': 0.02646126785926866, 'name': 'high commissions purpose'},
  {'confidence': 0.025153244745834618, 'name': 'diplomatic network services'},
  {'confidence': 0.021560783338911479, 'name': 'gratitude'},
  {'confidence': 0.01883305132877925, 'name': 'high commissions definiton'}],
 'text': 'nigger'}